In [1]:
import pandas as pd

In [2]:
df1 = pd.read_parquet('../data/jp_morgan/sorted/transaction_metrics_final_aml_v3_cbflag.parquet')
# aml_data = df1[['USD_amount', 'Label']].copy()
# aml_data.columns = ['amount', 'label']

df2 = pd.read_parquet('../data/jp_morgan/sorted/transaction_metrics_final_fraud_v3_cbflag.parquet')
# fraud_data = df2[['USD_amount', 'Label']].copy()
# fraud_data.columns = ['amount', 'label']

# df3 = pd.read_parquet('../data/jp_morgan/sorted/transaction_metrics_final_ds3_v3_cbflag.parquet')
# combined_data = df2[['USD_amount', 'Label']].copy()
# combined_data.columns = ['amount', 'label']

In [3]:
def calculate_modified_zscore(df, days_list):
    """
    Calculate modified z-score based on given conditions for multiple day periods
    
    Args:
        df: pandas DataFrame
        days_list: list of integers representing different day periods
    
    Returns:
        DataFrame with modified z-scores for each period
    """
    for days in days_list:
    # Calculate 75th percentile threshold
        p75 = df[f'velocity_{days}d_count'].quantile(0.75)
        
        # Initialize result series with zeros
        result = pd.Series(0, index=df.index)
        
        # Calculate modified z-score for rows where count > P75
        mask = df[f'velocity_{days}d_count'] > p75
        result[mask] = 0.6745 * (df.loc[mask, 'USD_amount'] - df.loc[mask, f'stat_{days}d_median']) / df.loc[mask, f'stat_{days}d_mad']
        
        # Assign result to new column
        df[f'stat_{days}d_modzscr'] = result
    
    return df

df1_mod = calculate_modified_zscore(df1, [7, 14, 30])
df2_mod = calculate_modified_zscore(df2, [7, 14, 30])
# df3_mod = calculate_modified_zscore(df3, [7, 14, 30])

C:\Users\dandr\AppData\Local\Temp\ipykernel_27228\427450559.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 6.32641624e+00 -6.77819363e-01  8.76192363e+04 ... -1.45392222e+00
 -6.73585506e-01 -2.02350000e+00]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result[mask] = 0.6745 * (df.loc[mask, 'USD_amount'] - df.loc[mask, f'stat_{days}d_median']) / df.loc[mask, f'stat_{days}d_mad']
C:\Users\dandr\AppData\Local\Temp\ipykernel_27228\427450559.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.81814596 -0.24609787  4.69447475 ... -1.0960625   0.57073077
  0.19741463]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result[mask] = 0.6745 * (df.loc[mask, 'USD_amount'] - df.loc[mask, f'stat_{days}d_median']) / df.loc[mask, f'stat_{day

In [4]:
df1_mod.to_parquet('../data/jp_morgan/sorted/transaction_metrics_final_aml_v4_cbflag_mod.parquet')
df2_mod.to_parquet('../data/jp_morgan/sorted/transaction_metrics_final_fraud_v4_cbflag_mod.parquet')
# df3_mod.to_parquet('../data/jp_morgan/sorted/transaction_metrics_final_ds3_v4_cbflag_mod.parquet')